In [22]:
import numpy as np
import torch
from PIL import Image
from torch.autograd import Variable

from face_detection.get_nets import PNet, RNet, ONet
from face_detection.box_utils import nms, calibrate_box, get_image_boxes, convert_to_square
from face_detection.first_stage import run_first_stage
from face_detection.visualization_utils import show_boxes

In [23]:
# Load Model
pnet = PNet()
rnet = RNet()
onet = ONet()
onet.eval()

ONet(
  (features): Sequential(
    (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1))
    (prelu1): PReLU(num_parameters=32)
    (pool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
    (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
    (prelu2): PReLU(num_parameters=64)
    (pool2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
    (conv3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
    (prelu3): PReLU(num_parameters=64)
    (pool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=True)
    (conv4): Conv2d(64, 128, kernel_size=(2, 2), stride=(1, 1))
    (prelu4): PReLU(num_parameters=128)
    (flatten): Flatten()
    (conv5): Linear(in_features=1152, out_features=256, bias=True)
    (drop5): Dropout(p=0.25)
    (prelu5): PReLU(num_parameters=256)
  )
  (conv6_1): Linear(in_features=256, out_features=2, bias=True)
  (conv6_2): Linear(in_features=256, out_features=4, bias=True)
  (co

In [24]:
# 设置超参数

# if the value is too low, the algorithm will use a lot of memory
min_face_size = 15.0

thresholds = [0.6, 0.7, 0.8]
nms_thresholds = [0.7, 0.7, 0.7]


In [25]:
# load an image
image = Image.open('images/office1.jpg')
image.show()

In [26]:
width, height = image.size
min_length = min(width, height)

min_detection_size = 12
factor = 0.707

scales = []

m = min_detection_size / min_face_size
min_length *= m

factor_count = 0
while min_length > min_detection_size:
    scales.append(m*factor**factor_count)
    min_length *= factor
    factor_count += 1
    
print("scales:", ['{:.2f}'.format(s) for s in scales])
print('number of different scales:', len(scales))



scales: ['0.80', '0.57', '0.40', '0.28', '0.20', '0.14', '0.10', '0.07', '0.05', '0.04']
number of different scales: 10


In [27]:
bounding_boxes = []

# 运行PNet
for s in scales:
    boxes = run_first_stage(image, pnet, scale=s, threshold=thresholds[0])
    bounding_boxes.append(boxes)

bounding_boxes = [i for i in bounding_boxes if i is not None]
bounding_boxes = np.vstack(bounding_boxes)
print('number of bounding_boxes:', len(bounding_boxes))

D:\pyCase\face_detection\face_detection\first_stage.py:32: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  img = Variable(torch.FloatTensor(_preprocess(img)), volatile=True)
D:\pyCase\face_detection\face_detection\get_nets.py:53: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  a = F.softmax(self.conv4_1(x))


number of bounding_boxes: 1021


In [28]:
show_boxes(image, bounding_boxes).show()

In [29]:
# NMS + calibration
keep = nms(bounding_boxes[:, 0:5], nms_thresholds[0])
bounding_boxes = bounding_boxes[keep]

# use offsets predicted by pnet to transform bounding boxes
bounding_boxes = calibrate_box(bounding_boxes[:, 0:5], bounding_boxes[:, 5:])
# shape[n_boxes, 5]

bounding_boxes = convert_to_square(bounding_boxes)
bounding_boxes[:, 0:4] = np.round(bounding_boxes[:, 0:4])
print('number of bounding boxes:', len(bounding_boxes))




















number of bounding boxes: 1021


In [30]:
show_boxes(image, bounding_boxes=bounding_boxes).show()

In [31]:
# RNet

img_boxes = get_image_boxes(bounding_boxes, image, size=24)
img_boxes = Variable(torch.FloatTensor(img_boxes), volatile=True)
output = rnet(img_boxes)
offsets = output[0].data.numpy()
probs = output[1].data.numpy()


D:\Anaconda\Anaconda\lib\site-packages\ipykernel_launcher.py:4: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  after removing the cwd from sys.path.


D:\pyCase\face_detection\face_detection\get_nets.py:91: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  a = F.softmax(self.conv5_1(x))


In [32]:
keep = np.where(probs[:, 1] > thresholds[1])[0]
bounding_boxes = bounding_boxes[keep]
bounding_boxes[:, 4] = probs[keep, 1].reshape((-1,))
offsets = offsets[keep]
print('number of bounding_boxes:', len(bounding_boxes))

number of bounding_boxes: 170


In [33]:
show_boxes(image, bounding_boxes).show()

In [34]:
# NMS + calibration
keep = nms(bounding_boxes, nms_thresholds[1])
bounding_boxes = bounding_boxes[keep]
bounding_boxes = calibrate_box(bounding_boxes, offsets[keep])
bounding_boxes = convert_to_square(bounding_boxes)
bounding_boxes[:, 0:4] = np.round(bounding_boxes[:, 0:4])
print('number of bounding_boxes:', len(bounding_boxes))

number of bounding_boxes: 93


In [35]:
show_boxes(image, bounding_boxes).show()

In [36]:

# ONet使用
img_boxes = get_image_boxes(bounding_boxes, image, size=48)
img_boxes = Variable(torch.FloatTensor(img_boxes), volatile=True)
output = onet(img_boxes)

landmarks = output[0].data.numpy()
offsets = output[1].data.numpy()
probs = output[2].data.numpy()




D:\Anaconda\Anaconda\lib\site-packages\ipykernel_launcher.py:4: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  after removing the cwd from sys.path.
D:\pyCase\face_detection\face_detection\get_nets.py:133: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  a = F.softmax(self.conv6_1(x))


In [37]:
keep = np.where(probs[:, 1] > thresholds[2])[0]

bounding_boxes = bounding_boxes[keep]
bounding_boxes[:, 4] = probs[keep, 1].reshape((-1,))
offsets = offsets[keep]
landmarks = landmarks[keep]

# compute landmark points
width = bounding_boxes[:, 2] - bounding_boxes[:, 0] + 1.0
height = bounding_boxes[:, 3] - bounding_boxes[:, 1] + 1.0
xmin, ymin = bounding_boxes[:, 0], bounding_boxes[:, 1]
landmarks[:, 0:5] = np.expand_dims(xmin, 1) + np.expand_dims(width, 1)*landmarks[:, 0:5]
landmarks[:, 5:10] = np.expand_dims(ymin, 1) + np.expand_dims(height, 1)*landmarks[:, 5:10]
print('number of bounding boxes:', len(bounding_boxes))

number of bounding boxes: 71


In [38]:
show_boxes(image, bounding_boxes, landmarks).show()

In [39]:

# nms + calibrate
bounding_boxes = calibrate_box(bounding_boxes, offsets)
keep = nms(bounding_boxes, nms_thresholds[2], mode='min')
bounding_boxes = bounding_boxes[keep]
landmarks = landmarks[keep]
print("number of bounding boxes:", len(bounding_boxes))


number of bounding boxes: 16


In [40]:
show_boxes(image, bounding_boxes, landmarks).show()